In [1]:
import pandas as pd
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
df_sample=pd.read_csv('sample_submission.csv')


In [33]:
df_train.head()

id  perc_premium_paid_by_cash_credit  age_in_days  Income  \
0  110936                             0.429        12058  355060   
1   41492                             0.010        21546  315150   
2   31300                             0.917        17531   84140   
3   19415                             0.049        15341  250510   
4   99379                             0.052        31400  198680   

   Count_3-6_months_late  Count_6-12_months_late  \
0                    0.0                     0.0   
1                    0.0                     0.0   
2                    2.0                     3.0   
3                    0.0                     0.0   
4                    0.0                     0.0   

   Count_more_than_12_months_late  application_underwriting_score  \
0                             0.0                           99.02   
1                             0.0                           99.89   
2                             1.0                           98.69   
3                             0.0                           99.57   
4                             0.0                           99.87   

   no_of_premiums_paid sourcing_channel residence_area_type  premium  
0                   13                C               Urban     3300  
1                   21                A               Urban    18000  
2                    7                C               Rural     3300  
3                    9                A               Urban     9600  
4                   12                B               Urban     9600

In [3]:
df_test.head()

id  perc_premium_paid_by_cash_credit  age_in_days  Income  \
0    649                             0.001        27384   51150   
1  81136                             0.124        23735  285140   
2  70762                             1.000        17170  186030   
3  53935                             0.198        16068  123540   
4  15476                             0.041        10591  200020   

   Count_3-6_months_late  Count_6-12_months_late  \
0                    0.0                     0.0   
1                    0.0                     0.0   
2                    0.0                     0.0   
3                    0.0                     0.0   
4                    1.0                     0.0   

   Count_more_than_12_months_late  application_underwriting_score  \
0                             0.0                           99.89   
1                             0.0                           98.93   
2                             0.0                             NaN   
3                             0.0                           99.00   
4                             0.0                           99.17   

   no_of_premiums_paid sourcing_channel residence_area_type  premium  
0                    7                A               Rural     3300  
1                   19                A               Urban    11700  
2                    2                B               Urban    11700  
3                   11                B               Rural     5400  
4                   14                A               Rural     9600

In [4]:
df_sample.head()

id  renewal  incentives
0    649      0.5        1650
1  81136      0.5        1650
2  70762      0.5        1650
3  53935      0.5        1650
4  15476      0.5        1650

In [5]:
df_train.isnull().sum()

id                                     0
perc_premium_paid_by_cash_credit       0
age_in_days                            0
Income                                 0
Count_3-6_months_late                 97
Count_6-12_months_late                97
Count_more_than_12_months_late        97
application_underwriting_score      2974
no_of_premiums_paid                    0
sourcing_channel                       0
residence_area_type                    0
premium                                0
renewal                                0
dtype: int64

In [6]:
df_train['Count_3-6_months_late'].fillna(0,inplace=True)
df_test['Count_3-6_months_late'].fillna(0,inplace=True)

In [7]:
df_train['Count_6-12_months_late'].fillna(0,inplace=True)
df_test['Count_6-12_months_late'].fillna(0,inplace=True)

In [8]:
df_train['Count_more_than_12_months_late'].fillna(0,inplace=True)
df_test['Count_more_than_12_months_late'].fillna(0,inplace=True)

In [9]:
df_train['application_underwriting_score'].fillna(df_train['application_underwriting_score'].mean(),inplace=True)
df_test['application_underwriting_score'].fillna(df_test['application_underwriting_score'].mean(),inplace=True)

In [10]:
renewal=df_train['renewal']
df_train=df_train.drop(['renewal'],axis=1)
df_total=pd.concat([df_train,df_test])



In [11]:
df_total=pd.get_dummies(df_total,columns=['sourcing_channel'],drop_first=True)
df_total=pd.get_dummies(df_total,columns=['residence_area_type'],drop_first=True)

In [12]:
shape=df_train.shape[0]

In [13]:
df_total.head()

id  perc_premium_paid_by_cash_credit  age_in_days  Income  \
0  110936                             0.429        12058  355060   
1   41492                             0.010        21546  315150   
2   31300                             0.917        17531   84140   
3   19415                             0.049        15341  250510   
4   99379                             0.052        31400  198680   

   Count_3-6_months_late  Count_6-12_months_late  \
0                    0.0                     0.0   
1                    0.0                     0.0   
2                    2.0                     3.0   
3                    0.0                     0.0   
4                    0.0                     0.0   

   Count_more_than_12_months_late  application_underwriting_score  \
0                             0.0                           99.02   
1                             0.0                           99.89   
2                             1.0                           98.69   
3                             0.0                           99.57   
4                             0.0                           99.87   

   no_of_premiums_paid  premium  sourcing_channel_B  sourcing_channel_C  \
0                   13     3300                   0                   1   
1                   21    18000                   0                   0   
2                    7     3300                   0                   1   
3                    9     9600                   0                   0   
4                   12     9600                   1                   0   

   sourcing_channel_D  sourcing_channel_E  residence_area_type_Urban  
0                   0                   0                          1  
1                   0                   0                          1  
2                   0                   0                          0  
3                   0                   0                          1  
4                   0                   0                          1

In [14]:
df_total=df_total.drop(['perc_premium_paid_by_cash_credit','age_in_days','application_underwriting_score'])

In [15]:
df_total_train=df_total.iloc[:shape]
df_total_test=df_total.iloc[shape:]
X=df_total_train.values
y=renewal.values
test=df_total_test.values

In [34]:
from sklearn.ensemble import RandomForestClassifier
y_pred_new_2=[]
rf = RandomForestClassifier(n_estimators = 10000,max_depth=8,random_state = 42)
rf.fit(X,y)
y_pred=rf.predict_proba(test)


KeyboardInterrupt: 

In [ ]:
for i in range(len(y_pred)):
    y_pred_new_2.append(y_pred[i][1])
    
df_test['renewal']=y_pred_new_2

In [27]:
df_test['incentives']=1500
df_test[['id','renewal','incentives']].to_csv('submission_2.csv',index=False)

In [28]:
df_result=pd.read_csv('submission_2.csv')
df_result

id   renewal  incentives
0         649  0.987206        1500
1       81136  0.981766        1500
2       70762  0.832261        1500
3       53935  0.974548        1500
4       15476  0.953272        1500
5       64797  0.979528        1500
6       67412  0.861003        1500
7       44241  0.848737        1500
8        5069  0.986811        1500
9       16615  0.987369        1500
10      84281  0.979767        1500
11      82623  0.966982        1500
12      86475  0.658446        1500
13      68263  0.948467        1500
14     113135  0.951793        1500
15      45532  0.974308        1500
16     102161  0.987338        1500
17      84789  0.716021        1500
18      62398  0.974169        1500
19      70148  0.972781        1500
20     102487  0.923735        1500
21     100513  0.908120        1500
22      92702  0.919932        1500
23      25051  0.678506        1500
24      92920  0.987357        1500
25      25520  0.965032        1500
26      81869  0.969021        1500
27      75402  0.975388        1500
28      34053  0.668603        1500
29      21191  0.980731        1500
...       ...       ...         ...
34194   79950  0.982804        1500
34195   30272  0.987026        1500
34196   85914  0.988824        1500
34197   10922  0.981443        1500
34198   93046  0.982921        1500
34199   52826  0.939891        1500
34200   35994  0.980124        1500
34201    1512  0.984055        1500
34202   52077  0.971022        1500
34203   95749  0.973245        1500
34204   49946  0.978704        1500
34205   66811  0.977721        1500
34206   53438  0.984031        1500
34207   24435  0.980057        1500
34208   17848  0.973907        1500
34209   53626  0.977527        1500
34210   87268  0.986831        1500
34211   58450  0.976736        1500
34212   48188  0.984663        1500
34213   44239  0.974753        1500
34214   24582  0.929389        1500
34215   79939  0.961719        1500
34216   63759  0.969652        1500
34217   63725  0.724266        1500
34218   37116  0.962180        1500
34219   81988  0.965689        1500
34220   30326  0.978176        1500
34221   65749  0.960186        1500
34222   65686  0.750272        1500
34223   93826  0.957004        1500

[34224 rows x 3 columns]